In [1]:
import face_recognition
import cv2
import numpy as np
import os
import re

In [2]:

known_faces_folder = "/home/hiren/Desktop/FaceRecognition/known_faces"

In [3]:
known_face_encodings = []
known_face_names = []

In [4]:
# Cleaning file names
def clean_name(filename):
    name = os.path.splitext(filename)[0]  # Remove file extension
    name = re.sub(r'\(\d+\)', '', name)
    name = re.sub(r'\d+$', '', name)

    return name.strip()

In [5]:
# Load known faces
for filename in os.listdir(known_faces_folder):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        img_path = os.path.join(known_faces_folder, filename)
        image = face_recognition.load_image_file(img_path)

        # Face encodings
        encodings = face_recognition.face_encodings(image)
        if encodings:
            known_face_encodings.append(encodings[0])
            known_face_names.append(clean_name(filename))  # Store cleaned name


Occupancy and Aiva Response Code

In [ ]:
import cv2
import face_recognition
import numpy as np
import time
from gtts import gTTS
import os
import pygame
import tempfile


pygame.mixer.init()


video_capture = cv2.VideoCapture(0)

# Track visitors
current_visitors = set()  # Names currently in frame
greeted_visitors = set()  # Names that have been greeted in this session
last_output_time = 0
output_interval = 5  # Minimum seconds between status outputs
last_status = None

def speak(text):
    """Convert text to speech using female American English voice"""
    try:
        tts = gTTS(text=text, lang='en', tld='us', slow=False)
        with tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as fp:
            temp_filename = fp.name
        tts.save(temp_filename)
        pygame.mixer.music.load(temp_filename)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)
        pygame.mixer.music.unload()
        os.unlink(temp_filename)
    except Exception as e:
        print(f"Speech synthesis error: {e}")

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Process frame
    small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    rgb_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Detect faces
    current_names = set()
    new_visitors = set()
    detected_names = []
    unknown_faces = 0
    
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        name = "Unknown"
        is_unknown = True

        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

        if True in matches:
            best_match_index = np.argmin(face_distances)
            if face_distances[best_match_index] < 0.5:
                name = known_face_names[best_match_index]
                is_unknown = False

        if is_unknown:
            unknown_faces += 1
        else:
            detected_names.append(name)
            current_names.add(name)
            if name not in current_visitors and name not in greeted_visitors:
                new_visitors.add(name)
                greeted_visitors.add(name)

        # Draw on frame
        top, right, bottom, left = top * 2, right * 2, bottom * 2, left * 2
        color = (0, 0, 255) if is_unknown else (0, 255, 0)
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Greet new visitors
    for name in new_visitors:
        greeting = f"Hi {name}, I am AIVA, your assistant. How can I help you?"
        print(f"AIVA: {greeting}")
        speak(greeting)

    # Update visitor tracking
    departed_visitors = current_visitors - current_names
    current_visitors = current_names

    # Clear greeted visitors if they leave
    for name in departed_visitors:
        if name in greeted_visitors:
            greeted_visitors.remove(name)

    # Print status only on significant changes
    current_time = time.time()
    current_status = {
        'occupancy': len(face_locations),
        'recognized': detected_names,
        'unknown': unknown_faces
    }

    should_print = False
    
    # Print if status changed significantly
    if current_status != last_status:
        should_print = True
    # Or if periodic interval passed and we have faces
    elif len(face_locations) > 0 and current_time - last_output_time > output_interval:
        should_print = True
    # Or if someone just left
    elif len(departed_visitors) > 0:
        should_print = True

    if should_print:
        last_output_time = current_time
        last_status = current_status
        
        if len(face_locations) == 0:
            print("\033[92mSTATUS: CLEAN - No faces detected\033[0m")
        else:
            print("Occupancy:", len(face_locations))
            print("recognized_faces:", len(detected_names))
            print("unknown_faces:", unknown_faces)
            print("recognized_names:", detected_names if detected_names else "None")
            print("timestamp:", time.strftime("%Y-%m-%d %H:%M:%S"))
            
            if unknown_faces > 0:
                print("\033[91mALERT: {} face(s) detected ({} unknown)!\033[0m".format(
                    len(face_locations), unknown_faces))
            else:
                print("\033[93mNOTICE: {} recognized face(s) detected\033[0m".format(len(detected_names)))

    # Display frame with status
    if len(face_locations) == 0:
        status_text = "Status: CLEAN - No faces detected"
        status_color = (0, 255, 0)
    else:
        status_text = f"Status: {len(face_locations)} face(s) detected"
        status_color = (0, 165, 255)
        if unknown_faces > 0:
            status_text += f" ({unknown_faces} unknown)"

    cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, status_color, 2)
    
    if detected_names:
        cv2.putText(frame, "Recognized: " + ", ".join(detected_names), (10, 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    elif len(face_locations) > 0:
        cv2.putText(frame, "No recognized faces", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow("Live Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
video_capture.release()
cv2.destroyAllWindows()
pygame.quit()

AIVA: Hi rini, I am AIVA, your assistant. How can I help you?
Occupancy: 1
recognized_faces: 1
unknown_faces: 0
recognized_names: ['rini']
timestamp: 2025-04-11 15:05:18
NOTICE: 1 recognized face(s) detected
STATUS: CLEAN - No faces detected
AIVA: Hi rini, I am AIVA, your assistant. How can I help you?
Occupancy: 1
recognized_faces: 1
unknown_faces: 0
recognized_names: ['rini']
timestamp: 2025-04-11 15:05:32
NOTICE: 1 recognized face(s) detected


Proper working code of Face Recognition Only

In [37]:

video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if not ret:
        break

    # Fame resizing faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)

    # Convert to RGB
    rgb_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Find faces and their encodings
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Process each detected face
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        name = "Unknown"
        color = (0, 0, 255)  # Default to red for "Unknown"

        # Compare with known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

        # If there is at least one match
        if True in matches:
            best_match_index = np.argmin(face_distances)  # Get the closest match

            # Set a confidence threshold (lower distance=better match)
            if face_distances[best_match_index] < 0.5:  # Adjust threshold if needed
                name = known_face_names[best_match_index]
                color = (0, 255, 0)  # green for known faces

        # Scale back up face locations since we resized the image
        top, right, bottom, left = top * 2, right * 2, bottom * 2, left * 2

        
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Display the frame
    cv2.imshow("Live Face Recognition", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()